In [1]:
import sys
sys.version_info

sys.version_info(major=2, minor=7, micro=10, releaselevel='final', serial=0)

In [2]:
import sklearn as sk
import pandas as pd
import numpy as np
import keras as ks

Using Theano backend.


In [55]:
## Load and clean the Iris data

from sklearn import datasets

iris = datasets.load_iris()

X = iris.data
Y = iris.target

df = pd.DataFrame(data = np.column_stack((X, Y)))
df.columns = ["Sepal length", "Sepal width", "Petal length", "Petal width", "Species"]
df["Species"] = df["Species"].astype('category')
df["Species"].cat.categories = iris.target_names

In [56]:
df.head()

,Sepal length,Sepal width,Petal length,Petal width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


TODO: NN, RF, SVM, and hyperparameter tuning therof with standard performance metrics for each